In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from bs4 import BeautifulSoup

from xgboost import XGBClassifier

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('./largest.csv', encoding='latin')

In [3]:
df.dropna(inplace=True)

In [4]:
df.head()

,Assigned_Group,ClosureProductName,IncidentProduct,Description,Custom_category
0,ITOPS-PLAN-L1,GLOVIA DAO/ASL (32681),GLOVIA DAO/ASL (32681),#Relief - Order missing relief in Glovia - Cos...,L1 App
1,ITOPS-TOOLS-L1,ORACLE DATABASE,ORACLE DATABASE,Request to remove the below Mentioned targets ...,L1 App
2,ITOPS-IEO ITRC Monitoring Network-Data,NETWORK DATA DEVICE,NETWORK DATA DEVICE,Network Outage_-_ _-_LCJ1MFGETL2K2,IT Ops L1 Monitoring Response Center
3,ITOPS-IEO ITRC Monitoring Database,Oracle Enterprise Manager (32850),Oracle Enterprise Manager (32850),Target Availability_-_Status - / -ausfloem2db0...,IT Ops L1 Monitoring Response Center
4,ITOPS-IEO ITRC Monitoring Database,SmartPrice (391181),SmartPrice (391181),Target Availability_-_Status - / -ausplsmphadb...,IT Ops L1 Monitoring Response Center


In [5]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

# Combining all the above stundents 


In [6]:
short_desc = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Description'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    short_desc.append(sentance.strip())

df['cleaned'] = short_desc

100%|██████████| 156351/156351 [00:58<00:00, 2675.41it/s]


In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(df['cleaned'], df['Assigned_Group'],
                                                    shuffle=False, test_size=0.2)

In [59]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
tf_idf_vect.fit(X_train)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

X_train_tf_idf = tf_idf_vect.transform(X_train)
print("the type of count vectorizer ",type(X_train_tf_idf))
print("the shape of out text TFIDF vectorizer ",X_train_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", X_train_tf_idf.get_shape()[1])

X_test_tfidf = tf_idf_vect.transform(X_test)

some sample features(unique words in the corpus) ['aa', 'aa cc', 'aa config', 'aaci', 'aaci dont', 'aalnskdkaeavk', 'aaplication', 'aarons', 'aarthi', 'aarthi ravivarman']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (125080, 95280)
the number of unique words including both unigrams and bigrams  95280


In [60]:
with open('tf_idf_vect_assigned_group_150k.pickle', 'wb') as f:
    pickle.dump(tf_idf_vect, f, pickle.HIGHEST_PROTOCOL)

In [61]:
clf = XGBClassifier()
clf.fit(X_train_tf_idf, Y_train)
clf.score(X_test_tfidf, Y_test)

0.7341626427041028

In [62]:
with open('clf_assigned_group_150k.pickle', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

In [79]:
review_text = ["mail"]

cleaned_review_text = []
for sentance in tqdm(review_text):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    cleaned_review_text.append(sentance.strip())


100%|██████████| 1/1 [00:00<00:00, 1888.48it/s]


In [80]:
z = tf_idf_vect.transform(cleaned_review_text)

In [81]:
clf.predict(z)

array(['ITOPS-SALES-L1'], dtype=object)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(df['cleaned'], df['ClosureProductName'], shuffle=False, test_size=0.2)

In [8]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
tf_idf_vect.fit(X_train)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

X_train_tf_idf = tf_idf_vect.transform(X_train)
print("the type of count vectorizer ",type(X_train_tf_idf))
print("the shape of out text TFIDF vectorizer ",X_train_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", X_train_tf_idf.get_shape()[1])

X_test_tfidf = tf_idf_vect.transform(X_test)

some sample features(unique words in the corpus) ['aa', 'aa cc', 'aa config', 'aaci', 'aaci dont', 'aalnskdkaeavk', 'aaplication', 'aarons', 'aarthi', 'aarthi ravivarman']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (125080, 95280)
the number of unique words including both unigrams and bigrams  95280


In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tf_idf, Y_train)
model_lr.score(X_test_tfidf, Y_test)